# Adjusting for cardholders

The method created so far is to have an adjustment table for each table and then this is read in and joined to the table that you are working with allowing you to create an adjusted spend/transcations value.

Further development will be required to see if we want to use one adjustment table or if we keep having seperate adjustment tables for each table. Moreover, how do we deal with different specifications, and the drop in cardholders over covid. These tables will have to be amended. 

The adjustment tables will be stored in fin_wip_notebook and read in accordingly. 


In [ ]:
project_path = "/home/jupyter"
import sys

sys.path.append(project_path)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t
from fintrans_toolbox.src import adjustments as adj

In [ ]:
client = bigquery.Client()
import importlib
importlib.reload(adj)


In [ ]:
# We first need to create our adjustment tables that we will bring in every time we want to create an adjusted value
# we do one for each table and each month/quarter then join on the date

# THE ADJUSTMENT TABLES will have to be amendended as they don't account for

In [ ]:
#test adjust rphst
df = t.read_retail_performance_high_streets_towns(
    client,
    time_period="Month",
    cardholder_location_level="POSTAL_AREA",
    merchant_location_level="All",
    mcg="All",
    cardholder_location="",
    merchant_location="",
)

df2 = adj.adjusted_rphst(df, "Month")
df2


In [ ]:
#Graph actual and adjusted spend
df3 = df2.loc[df2.cardholder_location == "LD"]
plt.plot(df3["date_time"],df3["spend"])
plt.plot(df3["date_time"], df3["idx_spend"])
plt.legend(loc='best')
plt.show()

In [ ]:
# Test SPOC
sql = "SELECT * FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` limit 500000 "
df = bq.read_bq_table_sql(client, sql)
client = bigquery.Client()

In [ ]:
# Two choices: Either specify cardholder origin parameter or keep the "cardholder_origin", "cardholder_origin_country" columns
df = df.loc[df.cardholder_origin == "UNITED KINGDOM"]
df1 = df.drop(columns =["cardholder_origin", "cardholder_origin_country"])

df2 = adj.adjusted_spoc(df1,cardholder_origin = "UNITED KINGDOM" )
df2

In [ ]:
# Two choices: Either specify cardholder origin parameter or keep the "cardholder_origin", "cardholder_origin_country" columns
df = df.loc[df.cardholder_origin == "UNITED KINGDOM"]

df3 = adj.adjusted_spoc(df)
df3

In [ ]:
# Test SML
sql = "SELECT * FROM `ons-fintrans-data-prod.fintrans_visa.spend_merchant_location` limit 10000"
client = bigquery.Client()
df = bq.read_bq_table_sql(client, sql)

In [ ]:
df_adj = adjusted_sml(df) 
df_adj